In [1]:
import h5py
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import numpy as np
import cv2
%matplotlib inline
import scipy

from itertools import cycle

import numpy as np
from scipy import ndimage
from scipy.ndimage import binary_dilation, binary_erosion
from msnake import MSnake

%load_ext autoreload
%autoreload 2
from tqdm import tqdm

In [14]:
# dice

PATH_MASK_H5 ="/mnt/dfs/xjyan/269-deeplevelset-segmentation/experiments/269_unet2/test/unet2_3_11_predictions.h5"
PATH_INPUT_H5 = "/mnt/dfs/xjyan/patches/269_test_patch_2class.h5"

msk_h5 = h5py.File(PATH_MASK_H5, "r")
input_h5 = h5py.File(PATH_INPUT_H5, "r")

l_dice = list()

for k in tqdm(list(np.random.choice(list(msk_h5.keys()), size=300, replace=False))):
    g = msk_h5[k]
    nn_mask_0 = g["binary_mask"][0,:,:,0] 
    nn_mask_1 = g["binary_mask"][0,:,:,1] 
    img = input_h5[k]["ct_slice"][()]
    mask = input_h5[k]["mask"][()]
    
    msnake_0 = MSnake(nn_mask_0, img, iterations=50)
    _ = msnake_0.evolve()
    msnake_1 = MSnake(nn_mask_1, img, iterations=50)
    _ = msnake_1.evolve()
    nn_mask = np.stack([msnake_0.mask, msnake_1.mask], axis=-1)
    
    l_dice.append(np.sum(nn_mask * mask) * 2/(np.sum(nn_mask) + np.sum(mask)))
    
print(np.average(l_dice))

 42%|████▏     | 125/300 [28:21<39:42, 13.61s/it] ../msnake.py:125: RuntimeWarning: invalid value encountered in true_divide
  c1 = self.img[inside].sum() / float(inside.sum())
../msnake.py:134: RuntimeWarning: invalid value encountered in less
  self.mask[aux < 0] = 1
../msnake.py:135: RuntimeWarning: invalid value encountered in greater
  self.mask[aux > 0] = 0
100%|██████████| 300/300 [1:16:06<00:00, 15.22s/it]

0.617267373267


In [3]:
np.stack([np.ones((3,3)), np.ones((3,3))], axis=-1).shape

(3, 3, 2)